In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'
!kaggle datasets download -d ananthu017/emotion-detection-fer

 92% 60.0M/65.2M [00:00<00:00, 220MB/s]
100% 65.2M/65.2M [00:00<00:00, 204MB/s]


In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content'
!kaggle datasets download -d gaborfodor/keras-pretrained-models

100% 941M/943M [00:09<00:00, 164MB/s]
100% 943M/943M [00:09<00:00, 108MB/s]


In [ ]:
!unzip keras-pretrained-models.zip

Archive:  keras-pretrained-models.zip
  inflating: Kuszma.JPG              
  inflating: imagenet_class_index.json  
  inflating: inception_resnet_v2_weights_tf_dim_ordering_tf_kernels.h5  
  inflating: inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5  
  inflating: inception_v3_weights_tf_dim_ordering_tf_kernels.h5  
  inflating: inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5  
  inflating: resnet50_weights_tf_dim_ordering_tf_kernels.h5  
  inflating: resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5  
  inflating: vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5  
  inflating: xception_weights_tf_dim_ordering_tf_kernels.h5  
  inflating: xception_weights_tf_dim_ordering_tf_kernels_notop.h5  


In [ ]:
!unzip emotion-detection-fer.zip

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

from IPython.display import Image, display

import matplotlib.pyplot as plt
import seaborn as sns

import cv2
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import random
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
from keras.layers import Activation,Dense,Flatten,BatchNormalization,Conv2D,MaxPool2D,Dropout,Input
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.optimizers import Adam
from keras import backend as K
from keras.applications import VGG16

import warnings
warnings.filterwarnings('ignore')

In [ ]:
!nvidia-smi

Fri Nov  4 10:04:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0    29W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
files = []
for dirname, _, filenames in os.walk('/content/train'):
    for filename in filenames:
        files.append(os.path.join(dirname, filename))

data = pd.DataFrame({'filename': files})
data['class'] = data['filename'].str.split('/').str[-2]
data = data.sample(frac=1)
data = data.reset_index(drop=True)
data.head()

,filename,class
0,/content/train/fearful/im1369.png,fearful
1,/content/train/fearful/im140.png,fearful
2,/content/train/happy/im711.png,happy
3,/content/train/neutral/im3500.png,neutral
4,/content/train/angry/im2366.png,angry


In [ ]:
data.shape

(28709, 2)

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
train, test = train_test_split(data, test_size=0.20)
train = train.reset_index(drop=True)
test = test.reset_index(drop=True)

train.shape, test.shape

((22967, 2), (5742, 2))

In [ ]:
train.head()

,filename,class
0,/content/train/happy/im1741.png,happy
1,/content/train/sad/im4088.png,sad
2,/content/train/happy/im1158.png,happy
3,/content/train/neutral/im768.png,neutral
4,/content/train/sad/im4372.png,sad


In [ ]:
datagen = ImageDataGenerator(validation_split=0.2, rescale=1./255)

train_generator = datagen.flow_from_dataframe(dataframe = train,
                                              directory = '/content/train',
                                              class_mode='categorical',
                                              batch_size = 32,
                                              target_size=(224, 224),
                                              subset='training',
                                              validate_filenames=False)

validation_generator = datagen.flow_from_dataframe(dataframe = train, 
                                                   directory = '/content/train',
                                                   class_mode='categorical',
                                                   batch_size = 32, 
                                                   target_size=(224, 224), 
                                                   subset='validation',
                                                   validate_filenames=False)

Found 18374 non-validated image filenames belonging to 7 classes.
Found 4593 non-validated image filenames belonging to 7 classes.


###VGG-16 Model

In [ ]:
tf.keras.backend.clear_session()

from keras.applications.vgg16 import VGG16

image_size = 224

weight_path = '/content/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'
vgg16_base_model = VGG16(weights=weight_path, 
                         include_top=False, 
                         input_shape=(image_size, image_size, 3))

for layer in vgg16_base_model.layers:
    layer.trainable = False

vgg16_pretrained_model = tf.keras.Sequential([
    vgg16_base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')
])
vgg16_pretrained_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
vgg16_pretrained_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dense_1 (Dense)             (None, 7)                 1799      
                                                                 
Total params: 21,139,271
Trainable params: 6,424,583
Non-trainable params: 14,714,688
_________________________________________________________________


In [ ]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=2)
history_vgg16 = vgg16_pretrained_model.fit(train_generator, epochs=10, callbacks=callback)

Epoch 1/10
575/575 [==============================] - 107s 164ms/step - loss: 1.5997 - accuracy: 0.4239
Epoch 2/10
575/575 [==============================] - 96s 167ms/step - loss: 1.2575 - accuracy: 0.5249
Epoch 3/10
575/575 [==============================] - 96s 168ms/step - loss: 1.1453 - accuracy: 0.5707
Epoch 4/10
575/575 [==============================] - 96s 168ms/step - loss: 1.0525 - accuracy: 0.6083
Epoch 5/10
575/575 [==============================] - 96s 168ms/step - loss: 0.9673 - accuracy: 0.6438
Epoch 6/10
575/575 [==============================] - 96s 168ms/step - loss: 0.8798 - accuracy: 0.6778
Epoch 7/10
575/575 [==============================] - 96s 167ms/step - loss: 0.8000 - accuracy: 0.7090
Epoch 8/10
575/575 [==============================] - 96s 168ms/step - loss: 0.7277 - accuracy: 0.7356
Epoch 9/10
575/575 [==============================] - 96s 167ms/step - loss: 0.6497 - accuracy: 0.7678
Epoch 10/10
575/575 [==============================] - 96s 167ms/step - 

# InceptionV3

In [ ]:
tf.keras.backend.clear_session()

from keras.applications.inception_v3 import InceptionV3

weight_path = '../content/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
inception_base_model = InceptionV3(weights=weight_path, 
                         include_top=False, 
                         input_shape=(image_size, image_size, 3))

for layer in inception_base_model.layers:
    layer.trainable = False

inception_pretrained_model = tf.keras.Sequential([
    inception_base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')
])
inception_pretrained_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
inception_pretrained_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 flatten (Flatten)           (None, 51200)             0         
                                                                 
 dense (Dense)               (None, 256)               13107456  
                                                                 
 dense_1 (Dense)             (None, 7)                 1799      
                                                                 
Total params: 34,912,039
Trainable params: 13,109,255
Non-trainable params: 21,802,784
_________________________________________________________________


In [ ]:
history_inception = inception_pretrained_model.fit(train_generator, epochs=10, callbacks=callback)

Epoch 1/10
575/575 [==============================] - 53s 81ms/step - loss: 2.1760 - accuracy: 0.3722
Epoch 2/10
575/575 [==============================] - 46s 80ms/step - loss: 1.4223 - accuracy: 0.4602
Epoch 3/10
575/575 [==============================] - 46s 80ms/step - loss: 1.3011 - accuracy: 0.5036
Epoch 4/10
575/575 [==============================] - 47s 81ms/step - loss: 1.1964 - accuracy: 0.5439
Epoch 5/10
575/575 [==============================] - 46s 79ms/step - loss: 1.1414 - accuracy: 0.5662
Epoch 6/10
575/575 [==============================] - 45s 79ms/step - loss: 1.0465 - accuracy: 0.5988
Epoch 7/10
575/575 [==============================] - 46s 79ms/step - loss: 0.9727 - accuracy: 0.6298
Epoch 8/10
575/575 [==============================] - 46s 80ms/step - loss: 0.9078 - accuracy: 0.6551
Epoch 9/10
575/575 [==============================] - 46s 81ms/step - loss: 0.8524 - accuracy: 0.6775
Epoch 10/10
575/575 [==============================] - 45s 79ms/step - loss: 0.786

In [ ]:
inception_pretrained_model.evaluate(validation_generator)

144/144 [==============================] - 14s 89ms/step - loss: 1.6848 - accuracy: 0.4703


[1.6847504377365112, 0.47028085589408875]

# Xception

In [ ]:
tf.keras.backend.clear_session()

from keras.applications.xception import Xception

weight_path = '../content/xception_weights_tf_dim_ordering_tf_kernels_notop.h5'
xception_base_model = Xception(weights=weight_path, 
                         include_top=False, 
                         input_shape=(image_size, image_size, 3))

for layer in xception_base_model.layers:
    layer.trainable = False

xception_pretrained_model = tf.keras.Sequential([
    xception_base_model,
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')
])
xception_pretrained_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
xception_pretrained_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 xception (Functional)       (None, 7, 7, 2048)        20861480  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 256)               25690368  
                                                                 
 dense_1 (Dense)             (None, 7)                 1799      
                                                                 
Total params: 46,553,647
Trainable params: 25,692,167
Non-trainable params: 20,861,480
_________________________________________________________________


In [ ]:
history_inception_xception = xception_pretrained_model.fit(train_generator, epochs=10, callbacks=callback)

Epoch 1/10
575/575 [==============================] - 89s 148ms/step - loss: 1.8905 - accuracy: 0.3874
Epoch 2/10
575/575 [==============================] - 84s 146ms/step - loss: 1.3442 - accuracy: 0.4843
Epoch 3/10
575/575 [==============================] - 84s 147ms/step - loss: 1.2125 - accuracy: 0.5340
Epoch 4/10
575/575 [==============================] - 84s 146ms/step - loss: 1.0972 - accuracy: 0.5790
Epoch 5/10
575/575 [==============================] - 84s 146ms/step - loss: 0.9929 - accuracy: 0.6244
Epoch 6/10
575/575 [==============================] - 84s 146ms/step - loss: 0.8729 - accuracy: 0.6714
Epoch 7/10
575/575 [==============================] - 84s 145ms/step - loss: 0.7759 - accuracy: 0.7076
Epoch 8/10
575/575 [==============================] - 84s 145ms/step - loss: 0.6954 - accuracy: 0.7397
Epoch 9/10
575/575 [==============================] - 84s 146ms/step - loss: 0.6003 - accuracy: 0.7776
Epoch 10/10
575/575 [==============================] - 84s 146ms/step - l

In [ ]:
xception_pretrained_model.evaluate(validation_generator)

144/144 [==============================] - 22s 148ms/step - loss: 2.1755 - accuracy: 0.4775


[2.1754977703094482, 0.4774657189846039]